In [13]:
import tensorflow as tf
import numpy as np

batch_size = 500
augmentation_time = 3
alpha = 0.5

sess = tf.InteractiveSession()

#入力値のplaceholderを作成（ラベル付きデータとラベル無しデータを分けて作成）
labeled_x = tf.placeholder(tf.float32, shape=(None, 28, 28, None))
unlabeled_x = tf.placeholder(tf.float32, shape=(None, 28, 28, None))
y = tf.placeholder(tf.float32, shape=(None,10))


#ラベル付きデータを-90°, -45°, 45°, 90°の間でランダムに回転（Augmentation）した画像を追加
extended_labeled_x = tf.concat([labeled_x, tf.contrib.image.rotate(labeled_x, np.random.choice([-90, -45, 45, 90]))], axis=0)

#ラベル無しデータを-90°, -45°, 45°, 90°の間でランダムに回転（Augmentation）した画像をaugmentation_timeセット追加
extended_unlabeled_x_set = [unlabeled_x]
for i in np.random.choice([-90, -45, 45, 90], augmentation_time):
  extended_unlabeled_x_set.append(tf.contrib.image.rotate(unlabeled_x, i))

#ラベル無しデータの各セットの予測値の平均を算出
average_prediction_unlabeled = tf.zeros([batch_size, 10], dtype=tf.float32)
extended_unlabeled_x = tf.zeros([0, 28, 28, 1], dtype=tf.float32)
counter = 0
for unlabeled_dataset in extended_unlabeled_x_set:
  input_layer = tf.reshape(unlabeled_dataset, [-1, 28, 28, 1])
  conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[4, 4], padding="same", activation=tf.nn.relu)
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2)
  conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.2)
  logits = tf.layers.dense(inputs=dropout, units=10)
  average_prediction_unlabeled = average_prediction_unlabeled + logits
  extended_unlabeled_x = tf.concat([extended_unlabeled_x, unlabeled_dataset], axis=0)
  counter += 1
average_prediction_unlabeled = average_prediction_unlabeled / counter
extended_prediction_unlabeled = tf.zeros([0, 10], dtype=tf.float32)
for i in range(augmentation_time+1):
  extended_prediction_unlabeled = tf.concat([extended_prediction_unlabeled, average_prediction_unlabeled], axis=0)

#ラベル無しデータの各セットの平均予測値をsharpeningする


#ラベル付きデータとラベル無しデータを行方向に連結し、シャッフルした行列Wを作成する
not_shuffled_W_x = tf.concat([extended_labeled_x, extended_unlabeled_x], axis=0)
not_shuffled_W_y = tf.concat([y, extended_prediction_unlabeled], axis=0)
shuffled_index = np.random.permutation(range(batch_size*(augmentation_time+1)))
indexes = tf.where(shuffled_index)
W_x, W_y = tf.gather_nd(not_shuffled_W_x, indexes), tf.gather_nd(not_shuffled_W_y, indexes)

#Mixupを実施
lam = np.random.beta(alpha, alpha)
mixed_labeled_x = lam*extended_labeled_x + (1-lam)*W_x[:batch_size*2]
mixed_labeled_y = lam*y + (1-lam)*W_y[:batch_size*2]
mixed_unlabeled_x = lam*extended_unlabeled_x + (1-lam)*W_x[batch_size*2:]
mixed_unlabeled_y = lam*extended_prediction_unlabeled + (1-lam)*W_y[:batch_size*2]

# ラベル付きデータの損失関数
input_layer = tf.reshape(mixed_labeled_x, [-1, 28, 28, 1])
conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[4, 4], padding="same", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2)
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.2)
logits = tf.layers.dense(inputs=dropout, units=10)
labeled_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = mixed_labeled_y) / tf.matrix_determinant(mixed_labeled_x)

# ラベル無しデータの損失関数
input_layer = tf.reshape(mixed_unlabeled_x, [-1, 28, 28, 1])
conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[4, 4], padding="same", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2)
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.2)
logits = tf.layers.dense(inputs=dropout, units=10)
unlabeled_loss = tf.reduce_sum(tf.square(mixed_unlabeled_y - logits)) / (tf.matrix_determinant(mixed_unlabeled_x)*y.shape[1])


# 最適化

#予測値・accuracyの算出

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: ignored

In [0]:
mnistのデータをロード
from keras.datasets import mnist 
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape([-1, 28, 28, 1]).astype(np.float32), x_test.reshape([-1, 28, 28, 1]).astype(np.float32)
y_train, y_test = np_utils.to_categorical(y_train, 10).astype(np.float32)[:30000], np_utils.to_categorical(y_test, 10).astype(np.float32)
labeled_x_train, unlabeled_x_train = x_train[:30000, :, :, :], x_train[30000:, :, :, :]

In [0]:
batch_size = 500
init = tf.global_variables_initializer()

with tf.Session() as sess:
  init.run()
  for num in range(len(labeled_x_train) // batch_size):
    labeled_x_batch, unlabeled_x_batch, y_batch = labeled_x_train[num*batch_size:(num+1)*batch_size], unlabeled_x_train[num*batch_size:(num+1)*batch_size], y_train[num*batch_size:(num+1)*batch_size]
    train_accuracy = accuracy.eval(feed_dict={labeled_x: labeled_x_batch, unlabeled_x: unlabeled_x_batch, y: y_batch})
    sess.run(train_op, feed_dict={labeled_x: labeled_x_batch, unlabeled_x: unlabeled_x_batch, y: y_batch})
  print('accuracy: {}'.format(train_accuracy))

### ラベル付きの訓練データ（30000件）のみで訓練するモデル

In [0]:
#入力値のplaceholderを作成
labeled_x = tf.placeholder(tf.float32, shape=(None, 28, 28, None))
y = tf.placeholder(tf.float32, shape=(None,10))

input_layer = tf.reshape(labeled_x, [-1, 28, 28, 1])
conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[4, 4], padding="same", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.2)
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.2)
logits = tf.layers.dense(inputs=dropout, units=10)


# 損失関数
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)

# 最適化
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

prediction_match = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name='accuracy')

In [0]:
batch_size = 500
init = tf.global_variables_initializer()

with tf.Session() as sess:
  init.run()
  for num in range(len(labeled_x_train) // batch_size):
    labeled_x_batch, unlabeled_x_batch, y_batch = labeled_x_train[num*batch_size:(num+1)*batch_size], unlabeled_x_train[num*batch_size:(num+1)*batch_size], y_train[num*batch_size:(num+1)*batch_size]
    train_accuracy = accuracy.eval(feed_dict={labeled_x: labeled_x_batch, unlabeled_x: unlabeled_x_batch, y: y_batch})
    sess.run(train_op, feed_dict={labeled_x: labeled_x_batch, unlabeled_x: unlabeled_x_batch, y: y_batch})
  print('accuracy: {}'.format(train_accuracy))

accuracy: 0.9399999976158142
